# RNN做图像分类

In [11]:
import sys
sys.path.append('..')
sys.path.append('../4-CNN')#把4-CNN中的utils加入环境变量
import torch
from torch.autograd import Variable
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms as tfs
from torchvision.datasets import MNIST

In [3]:
data_tf=tfs.Compose([tfs.ToTensor(),
                    tfs.Normalize([0.5],[0.5])])
train_set=MNIST('../data',train=True,transform=data_tf)
test_set=MNIST('../data',train=False,transform=data_tf)
train_data=DataLoader(train_set,64,True,num_workers=2)
test_data=DataLoader(test_set,128,False,num_workers=2)

In [15]:
#定义模型
class rnn_classify(nn.Module):
    def __init__(self,in_feature=28,hidden_feature=100,num_class=10,num_layers=2):
        super(rnn_classify,self).__init__()
        self.rnn=nn.LSTM(in_feature,hidden_feature,num_layers)#使用两层lstm
        self.classifier=nn.Linear(hidden_feature,num_layers)
    def forward(self,x):
        x=x.squeeze()# 去掉 (batch, 1, 28, 28) 中的 1，变成 (batch, 28, 28)
        x=x.permute(2,0,1) #注意,只要使用rnn,就必须有这一步.(28,batch,28)
        out,_=self.rnn(x) #使用默认的初始隐藏状态,得到的out是[28, batch, hidden_feature]
        out=out[-1,:,:] # 取序列中的最后一个，大小是 (batch, hidden_feature)
        out=self.classifier(out)# 得到分类结果
        return out

In [16]:
net=rnn_classify()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters(),lr=1e-1,weight_decay=1e-4)

In [17]:
from utils import train
train(net,train_data,test_data,10,optimizer,criterion)

Epoch 0. Train Loss: 0.661712, Train Acc: 0.162713, Valid Loss: 0.489222, Valid Acc: 0.205597, Time 00:00:07
Epoch 1. Train Loss: 0.637440, Train Acc: 0.200843, Valid Loss: 0.485146, Valid Acc: 0.207872, Time 00:00:07
Epoch 2. Train Loss: 0.634826, Train Acc: 0.204824, Valid Loss: 0.475037, Valid Acc: 0.209059, Time 00:00:07
Epoch 3. Train Loss: 0.629538, Train Acc: 0.207856, Valid Loss: 0.472900, Valid Acc: 0.210344, Time 00:00:07
Epoch 4. Train Loss: 0.626713, Train Acc: 0.208872, Valid Loss: 0.476694, Valid Acc: 0.210146, Time 00:00:07
Epoch 5. Train Loss: 0.622990, Train Acc: 0.209488, Valid Loss: 0.469965, Valid Acc: 0.210443, Time 00:00:07
Epoch 6. Train Loss: 0.621490, Train Acc: 0.209638, Valid Loss: 0.467361, Valid Acc: 0.210344, Time 00:00:07
Epoch 7. Train Loss: 0.619461, Train Acc: 0.209855, Valid Loss: 0.468658, Valid Acc: 0.209850, Time 00:00:07
Epoch 8. Train Loss: 0.618508, Train Acc: 0.209838, Valid Loss: 0.464815, Valid Acc: 0.210542, Time 00:00:07
Epoch 9. Train Loss